# KNN
Using two possible distance measures and rangeof k from 1 to 20 ( maybe can check overfitting with decreasing K)
try without standardization and try with mi-max (maybe try with Emmanuel's standardization)

In [2]:
import numpy as np
import csv

## Importing train data and organizing

In [3]:
dtypes = "i8,S5,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,i8,f8,f8,f8,f8,f8,f8,f8"
with open('train.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    headerstrain = next(reader)
datatrain = np.genfromtxt('train.csv', delimiter=",",names=True, dtype=dtypes)

#training (s=1 b=0)
y=(datatrain['Prediction']==b's').astype(int)
#transforming features X into a list of list
trainxlist=[]
for i in datatrain:
    trainsublist=[]
    for x in i:
        trainsublist.append(x)
    trainsublist=trainsublist[2:]
    trainxlist.append(trainsublist)

#adding column of 1 
for x in trainxlist:
    x.insert(0,1)
#trainxlist is the extended X matrix

#transform into np.array
Y_total=np.array(y)
X_total=np.array(trainxlist)

## OHE
Before comparing distances it doesn't make sense to keep categorical variabes as integers instead OHE them. From EDA 3 variabes could be split and considered as categorical. PRI_jet_num definitely since there are only 4 unique values. And the centrality look like they could be split into two distributions with the threshold below.

DER_lep_eta_cenrality: nans, 0, 1 threshold 0.5 for 0 vs 1
DER_met_phi_centrality: threshold above or below 0


In [ ]:
#column indices. The headers as two extra coumn 'id' and 'predictions', X matrices have only 1 extra column of ones so indices should be 1 less than in the headers 
indice_OHE=[]
indice_OHE.append(headerstrain.index('DER_prodeta_jet_jet')-1)
indice_OHE.append(headerstrain.index('DER_prodeta_jet_jet')-1)
indice_OHE.append(headerstrain.index('PRI_jet_num')-1)
cat=[2,2,4]


X_OHE=np.copy(X_total)  #X is the extanded "30+1 column array"

#JetNumOHE
colbefore=X_total[:,indice_OHE[0]] #extract column from Xtotal
classes=[0,1,2,3]
#initialize ohe sub-matrix for feature col
submatrix=np.zeros((X_total.shape[0], 4))
for i in range(4):
    coltmp=(colbefore==classes[i]).astype(int)
    submatrix[:,i]=coltmp
    
X_OHE=np.append(X_OHE, submatrix, axis=1)
    #remove column from XOHE
    #append new columns to XOHE

#NOTE: ONLY for X_Cleaned when no NAN values for the following two features. Are these two features retained with the threshold 
#LepEtaCentrality OHE
colbefore=X_total[:,indice_OHE[1]]
submatrix=np.zeros((X_total.shape[0], 2))
coltmp0=(colbefore < 0.5).astype(int)
coltmp1=(colbefore >= 0.5).astype(int)
submatrix[:,0]=coltmp0
submatrix[:,1]=coltmp1
X_OHE=np.append(X_OHE, submatrix, axis=1)

#LepEtaCentrality OHE
colbefore=X_total[:,indice_OHE[2]]
submatrix=np.zeros((X_total.shape[0], 2))
coltmp0=(colbefore < 0).astype(int)
coltmp1=(colbefore >= 0).astype(int)
submatrix[:,0]=coltmp0
submatrix[:,1]=coltmp1
X_OHE=np.append(X_OHE, submatrix, axis=1)
    

#removing old features
    
X_OHE=np.delete(X_OHE, indice_OHE, axis=1)

##  Scaling non-categorical feature 
so that distances are comparable

In [ ]:
Xscale=np.copy(XOHE)
Original_number=31  #number of features before OHE +1 column
number_feature_OHE=3 #again if the centrality features are still part of the X data array
newindex=Original_number-number_features_OHE
submatrix=Xscale[:,1:newindex]  #don't scale the 1 column and the OHE variables which start at index 31-3 because we removed 3 categorical variables
Xmin=(submatrix.min(axis=0)).reshape(-1,1) #minimum array for the different features
Xmax=(submatrix.max(axis=0)).reshape(-1,1) #maximum array for the different features
subscale=(submatrix-Xmin)/(Xmin-Xmax)
Xscale[1:newindex]=subscale

In [16]:
#pull maybe 1000 random samples
# Euclidean distance between two vectors
def dist_euclidean(row1, row2):
    r1=row1[1:] #takes features after column of 1
    r2=row2[1:]
    dist=np.linalg.norm(r1-rb)
    return dist

# calculate the manhattan distance between two vectors
def dist_manhattan(row1, row2):
    r1=row1[1:] #takes features after column of 1
    r2=row2[1:]
    dist=np.sum(np.abs(r1-r2))
    return dist

## KNN model

In [26]:
def KNNmodel(Xtest,Xtrain,Ytrain,dist_func, k_n):
    """ input are extended X matrices, returns ypredictions array for Xtest using k closest distances
    calculated with given dist_func Given X's have been OHE and scales and y predictions are given as 0/1"""
    ypredarray=np.zeros(Xtest.shape[0])
    for j in range(Xtest.shape[0]): #prediction for j'th row of Xtest
        rowdistances=np.zeros(Xtrain.shape[0])
        for i in range(Xtrain.shape[0]):   #i denotes index of row of Xtrain
            disttmp=dist_func(Xtest[j],Xtrain[i])  #computing distance between test point and each Xtrain row
            rowdistances[i]=disttmp                  #storing distance

        sortindex=np.argsort(rowdistances) #orders neighbor rows for closest distances 
        indexn=sortindex[:k_n]
        yneighbours=Ytrain[indexn]
        ypredclass=( yneighbours.mean()>0.5).astype(int)  #if average above 0.5 majority 1's classified as 1 otherwise classified as 0
        ypredarray[j]=ypredclass
    
    return ypredarray
        
    
    
    

NOTE
would it be possible to also (instead of splitting NAN - Non NANS and cleaning NAN's) Directly take X_total and clean NANs: remove features if more than 50% are NANs otherwise replace by median. 